In [0]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 4 - Tabelas tópico 4.3

### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.faixaScoreExterno':["R10"],
                             'payloadHomol.intermediarias.segmentacaoPolitica':["P1 - Funcionarios"],
                             'payloadHomol.intermediarias.faixaRendaLiquida':["d. >= 10k"],
                             'payloadHomol.intermediarias.admissaoMeses':[0],
                             'payloadHomol.intermediarias.cpf5e6Digito':[99]
                            })

In [0]:
#Criação das massas com casos forçados
massa_TabelaMarAberto = sherlock.Gera_Massa(base_entrada)
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.faixaScoreExterno',["R1","R2","R3",
                                                                                            "R4","R5","R6",
                                                                                            "R7","R8","R9",
                                                                                            "R10"])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.segmentacaoPolitica',["N1 - FOPAG",
                                                                                      "N2.1 - Varejo",
                                                                                      "N2.2 - Varejo+",
                                                                                      "N2.3 - Alta Renda",
                                                                                      "N2.1 - Varejo"])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.faixaRendaLiquida',['a. Sem renda',
                                                                                  'b. < 4k',
                                                                                  'c. 4k - 10k',
                                                                                  'd. >= 10k',
                                                                                  'z. Verificar'])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.admissaoMeses',[0,1,2,3,4,5])
massa_TabelaMarAberto.gera_cenario('payloadHomol.intermediarias.cpf5e6Digito',[0,43,44,45,46,88,89,90,91,99])


massa_TabelaMarAberto.get_massa_final()

massa_TabelaMarAberto.set_bom_arqv()
massa_TabelaMarAberto.converte_massa()

In [0]:
massa_TabelaMarAberto.get_massa_final()

### Execução Modular e Geração de CSV

In [0]:
#EXEC MULTI NUCLEOS
#listaResultado = rp.execucaoModularConcurrent(massa_TabelaMarAberto.get_massa_json(),rp.SubGruposMarAberto)

In [0]:
#EXEC NORMAL
listaResultado = rp.execucaoModular(massa_TabelaMarAberto.get_massa_json(),rp.SubGruposMarAberto)

In [0]:
print(json.dumps(listaResultado[0],indent = 4))

##### Valida se todas condições estão sendo abordadas

In [0]:
# buscando um perfil de cliente para análise
for payload in listaResultado:
    if (payload['payloadHomol']['intermediarias']['faixaScoreExterno']  == "R1"
        and
        payload['payloadHomol']['intermediarias']['segmentacaoPolitica'] == "N1 - FOPAG"
        and 
        payload['payloadHomol']['intermediarias']['faixaRendaLiquida'] == "b. < 4k"
        and
        payload['payloadHomol']['intermediarias']['admissaoMeses'] == 1
        ):
        print(json.dumps(payload,indent = 4))
        break

In [0]:
# mudando o max de display de linhas dos dataframes desse notebook
pd.set_option('display.max_rows', 300) 
import pandas as pd

# fazendo freq para analisar se todos os cenários da tabela são abordados na massa
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['cenario'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha.sort_values(by='valor', ascending=True)

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_TabelaMarAberto.rename_cabecalho()

In [0]:
massa_TabelaMarAberto.get_massa_dm()

In [0]:
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacao","segmentacaoSubgrupo")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.alavancagem","alavancagem")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteTeto","limiteTeto")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.limitePiso","limitePiso")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteFixo","limiteFixo")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoBAUAux","aprovadoBAUAux")
massa_TabelaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoSLAux","aprovadoSmallLimitsAux")

massa_TabelaMarAberto.get_massa_dm()

### Conversão da massa para CSV

In [0]:
massa_TabelaMarAberto.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaDefinicaoTabelasSubGruposMarAberto.csv',index=False)